In [7]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [8]:
import pandas as pd
import math
import keras
from keras.layers import Dense,Embedding, Flatten, Conv1D, GlobalMaxPooling1D, LSTM, Bidirectional, Dropout
from keras.preprocessing.text import text_to_word_sequence,Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from new_train_val_test import cc_train_val_test

In [9]:
train,val,test = cc_train_val_test.get_upsampled_raw_dataset()

In [13]:
train.clue = train.clue.apply(lambda x:list(x))
val.clue = val.clue.apply(lambda x:list(x))
test.clue = test.clue.apply(lambda x:list(x))

In [15]:
tokenizer = Tokenizer(filters="")

In [16]:
tokenizer.fit_on_texts(pd.concat([train,val,test]).clue)

In [21]:
train_x = train.clue
val_x = val.clue
test_x = test.clue

In [36]:
train_x = pad_sequences(tokenizer.texts_to_sequences(train_x),50)
val_x = pad_sequences(tokenizer.texts_to_sequences(val_x),50)
test_x = pad_sequences(tokenizer.texts_to_sequences(test_x),50)

In [37]:
train_y = train[train.columns[4:]]
val_y = val[train.columns[4:]]
test_y = test[test.columns[4:]]

In [38]:
filepath="./models/1xBilstm-{epoch:02d}-{loss:.2f}-{categorical_accuracy:.2f}-{val_loss:.2f}-{val_categorical_accuracy:.2f}-singlelabel.hdf5"
saveModelCallBack = keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0, write_graph=True, write_images=True)
from keras.callbacks import Callback

class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=0)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))

callbacks_list = [saveModelCallBack,tbCallBack,TestCallback((test_x, test_y))]


In [39]:
model = keras.Sequential()
model.add(Embedding(len(tokenizer.index_word)+1, 128))
model.add(Bidirectional(LSTM(128, dropout=0.5)))
model.add(Dense(12, activation='softmax'))

In [40]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['categorical_accuracy'])

In [41]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, None, 128)         9344      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_2 (Dense)              (None, 12)                3084      
Total params: 275,596
Trainable params: 275,596
Non-trainable params: 0
_________________________________________________________________


In [42]:
train_y.shape

(177792, 12)

In [43]:
test_y.shape

(9404, 12)

In [ ]:
history = model.fit(train_x,train_y ,validation_data=(test_x,test_y), batch_size=5000, epochs=15,shuffle=True,callbacks=callbacks_list,initial_epoch=1)


Train on 177792 samples, validate on 9404 samples
Epoch 2/15
 10000/177792 [>.............................] - ETA: 4:14 - loss: 1.4736 - categorical_accuracy: 0.4446

KeyboardInterrupt: 

In [ ]:
def simple_model():                                           
    model = keras.Sequential()
    model.add(Embedding(len(tokenizer.index_word)+1, 128,input_length=15))
    model.add(LSTM(128, dropout=0.5))
    model.add(Dense(1, activation='relu'))
    model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [ ]:
model = simple_model()
model.fit(train_data,train_data_out,batch_size=1024)

In [239]:
model.evaluate(test_data,test_data_out)

3138/3138 [==============================] - 2s 482us/step


[1.9973295485616263, 0.18387507966857872]

In [293]:
from sklearn.ensemble import AdaBoostRegressor

In [294]:
from keras.wrappers.scikit_learn import KerasRegressor

In [295]:
bilstm_estimator = KerasRegressor(build_fn= simple_model, epochs=1, batch_size=1024, verbose=1)

In [297]:
import numpy as np
# train_data_out_labels = train_data_out.values.argmax(axis=1)
boosted_lstm = AdaBoostRegressor(base_estimator= bilstm_estimator)
boosted_lstm.fit(train_data, train_data_out)# scale your training data 

Epoch 1/1
28187/28187 [==============================] - 3s 112us/step
Epoch 1/1
28187/28187 [==============================] - 3s 116us/step
Epoch 1/1
28187/28187 [==============================] - 3s 119us/step
Epoch 1/1
28187/28187 [==============================] - 3s 121us/step
Epoch 1/1
28187/28187 [==============================] - 4s 127us/step
Epoch 1/1
28187/28187 [==============================] - 4s 126us/step
Epoch 1/1
28187/28187 [==============================] - 4s 125us/step
Epoch 1/1
28187/28187 [==============================] - 4s 130us/step
Epoch 1/1
28187/28187 [==============================] - 4s 131us/step
Epoch 1/1
28187/28187 [==============================] - 4s 131us/step


AdaBoostRegressor(base_estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x18a6db8d0>,
         learning_rate=1.0, loss='linear', n_estimators=50,
         random_state=None)

In [298]:
boosted_lstm.score(test_data,test_data_out)

3138/3138 [==============================] - 0s 74us/step


-3.974468899422262

In [215]:
train_data_out_labels = [np.argmax(row).astype(int) for row in train_data_out.values]

In [218]:
type(train_data_out_labels.astype(int))

AttributeError: 'list' object has no attribute 'astype'